# Hyper Parameter Tunning

In [1]:
# standerd imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge
from sklearn.svm import LinearSVC
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import pickle as pc
%matplotlib inline

In [2]:
clf = RandomForestClassifier()
clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

## hyper Parameters
### 1, Mannually
### 2. Randomly with RandomSearchCV
### 3. Exhaustively GridSearchCV

In [3]:
# paramertes we are gong to use
# max_depth
# Max_features
# min_samples_split
# min_samples_leaf
# n_estimators

# Task_1 Divide Data into Train, test, Validate and predict

In [4]:
# Task_1
# Data
df1 = pd.read_csv('heart.csv')
df = df1.sample(frac=1)
# Divide data
X = df.drop('output', axis = 1)
y = df['output']
# Split data
train_split = round(.7*len(df))
valid_split = round(train_split + .15*len(df))
X_train, y_train = X[:train_split], y[:train_split]
X_valid, y_valid = X[train_split:valid_split], y[train_split:valid_split]
X_test, y_test = X[valid_split:], y[valid_split:]


In [5]:
len(df)

303

In [6]:
len(X_train), len(X_valid), len(X_test)

(212, 45, 46)

# Create function to evaluate every Model

In [7]:
def evalute_hyper_met(y_true, y_predicted):
    accuracy = accuracy_score(y_true,y_predicted)
    precision = precision_score(y_true,y_predicted)
    recall = recall_score(y_true,y_predicted)
    f_1 = f1_score(y_true,y_predicted)
    # return these
    metri_dic = {'Accuracy': accuracy,
                'Precision': precision,
                'recall': recall,
                'f1': f_1}
    print(f'Accuracy is {round(accuracy,2)}')
    print(f'Precision is {round(precision,2)}')
    print(f'Recall is {round(recall,2)}')
    print(f'F1 is {round(f_1,2)}')
    return metri_dic

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
clf = RandomForestClassifier().fit(X_train, y_train)
y_predict = clf.predict(X_valid)
y_predict
# Evaluating Mdodel 
first_result = evalute_hyper_met(y_valid, y_predict)

Accuracy is 0.87
Precision is 0.86
Recall is 0.92
F1 is 0.89


In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
clf2 = RandomForestClassifier(n_estimators=1000).fit(X_train, y_train)
y_predict = clf2.predict(X_valid)
y_predict
# Evaluating Mdodel 
second_result = evalute_hyper_met(y_valid, y_predict)

Accuracy is 0.84
Precision is 0.83
Recall is 0.92
F1 is 0.87


In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
clf3 = RandomForestClassifier(n_estimators=200, max_depth= 100).fit(X_train, y_train)
y_predict = clf3.predict(X_valid)
y_predict
# Evaluating Mdodel 
third_result = evalute_hyper_met(y_valid, y_predict)

Accuracy is 0.84
Precision is 0.85
Recall is 0.88
F1 is 0.87


# RSCV (Random Search CV)

In [11]:
# Randomized search Cross Validation
clf.get_params
# max_depth [None, 5, 10, 20, 30, 50]
# n_estimators [10, 100, 200, 500, 1000, 12000]
# min_samples_leaf  [1, 2, 3, 4, 5, 6]
# min_samples_split [2, 4, 6, 8, 10, 12] 
# max_features ['auto', 'sqrt']
# n_jobs

<bound method BaseEstimator.get_params of RandomForestClassifier()>

In [12]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Dictionary 
hparams = {'max_depth': [None, 5, 10, 20, 30],
           'n_estimators': [10, 100, 200, 500, 1000, 1200, 1500],
           'min_samples_leaf':  [1, 2, 3],
           'min_samples_split': [2, 4, 6],
           'max_features': ['sqrt', 'log2', None, 0.5, 10]
           }

# Data split 
X = df.drop('output', axis = 1)
y = df['output']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.20)

clf_h = RandomForestClassifier(n_jobs=1)
rscv_clf_h = RandomizedSearchCV(estimator=clf_h, param_distributions=hparams, n_iter=10, cv=5, verbose=2)
rscv_clf_h.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=1000; total time=   1.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=1000; total time=   1.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=1000; total time=   1.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=1000; total time=   1.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=1000; total time=   1.1s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=3, min_samples_split=2, n_estimators=1000; total time=   1.1s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=3, min_samples_split=2, n_estimators=1000; total time=   1.1s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=3, min_samples_split=2,

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(n_jobs=1),
                   param_distributions={'max_depth': [None, 5, 10, 20, 30],
                                        'max_features': ['sqrt', 'log2', None,
                                                         0.5, 10],
                                        'min_samples_leaf': [1, 2, 3],
                                        'min_samples_split': [2, 4, 6],
                                        'n_estimators': [10, 100, 200, 500,
                                                         1000, 1200, 1500]},
                   verbose=2)

In [13]:
rscv_clf_h.best_params_

{'n_estimators': 1500,
 'min_samples_split': 4,
 'min_samples_leaf': 3,
 'max_features': 'sqrt',
 'max_depth': None}

In [14]:
# running predictions
rscv_pred  = rscv_clf_h.predict(X_test)
# Evaluate predictions
rscv_result = evalute_hyper_met(y_test, rscv_pred)

Accuracy is 0.87
Precision is 0.96
Recall is 0.79
F1 is 0.87


# GRID Search Validation GSCV

In [15]:
from sklearn.model_selection import GridSearchCV
# dictionay 
hparams2 = {'max_depth': [5, 10,20],
           'n_estimators': [1000, 1100, 1200],
           'min_samples_leaf':  [2,3,4],
           'min_samples_split': [2, 3],
           'max_features': ['log2'],
           }
df = pd.read_csv('heart.csv')
# Data split 
X = df.drop('output', axis = 1)
y = df['output']
X_test, X_train, y_test, y_train = train_test_split(X, y, test_size= 0.20)

clf_h = RandomForestClassifier(n_jobs=1)
gscv_clf_h2 = GridSearchCV(estimator=clf_h, param_grid = hparams2, cv = 5, verbose=2)

In [16]:
gscv_clf_h2.fit(X_train, y_train)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV] END max_depth=5, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time=   1.0s
[CV] END max_depth=5, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time=   1.0s
[CV] END max_depth=5, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time=   1.0s
[CV] END max_depth=5, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time=   1.0s
[CV] END max_depth=5, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time=   1.0s
[CV] END max_depth=5, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=1100; total time=   1.1s
[CV] END max_depth=5, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=1100; total time=   1.1s
[CV] END max_depth=5, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=110

GridSearchCV(cv=5, estimator=RandomForestClassifier(n_jobs=1),
             param_grid={'max_depth': [5, 10, 20], 'max_features': ['log2'],
                         'min_samples_leaf': [2, 3, 4],
                         'min_samples_split': [2, 3],
                         'n_estimators': [1000, 1100, 1200]},
             verbose=2)

In [19]:
# running predictions
gscv_pred  = gscv_clf_h2.predict(X_test)
# Evaluate predictions
last_result = evalute_hyper_met(y_test, rscv_pred)

ValueError: Found input variables with inconsistent numbers of samples: [242, 61]

# Data Visulization

In [20]:
# Comparison with plot
compmarison = pd.DataFrame({
    "First_result": first_result,
    "Second_result": second_result,
    "Third_result" : third_result,
    "RSCV_result" : rscv_result,
    "GSCV_result" : last_result
})

NameError: name 'last_result' is not defined

In [ ]:
compmarison.plot.bar(figsize = (10,8))

In [ ]:
# saving Model
pc.dump(rscv_clf_h,open('RSCV_CLF_RFM.pkl','wb'))
pc.dump(gscv_clf_h2,open('GSCV_CLF_RFM.pkl','wb'))

In [ ]:
#Load_saved_model
picle_m = pc.load(gscv_clf_h2,open('GSCV_CLF_RFM.pkl','rb'))
